In [48]:
import sqlite3
import pandas as pd

!pip install ipython-sql



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
# Load the SpaceX dataset from IBM cloud storage
df = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"
)



In [50]:
# Create a SQLite database connection
con = sqlite3.connect("my_data1.db")

In [51]:
# Store the DataFrame as a SQL table for analysis 
df.to_sql("SPACEXTBL", con, if_exists="replace", index=False)


101

In [52]:
df.columns

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='object')

In [53]:
# Count the total number of records in the table
pd.read_sql("SELECT COUNT(*) FROM SPACEXTBL", con)


,COUNT(*)
0,101


In [54]:
# Retrieve all unique launch sites used in SpaceX missions
pd.read_sql(
    "SELECT DISTINCT Launch_Site FROM SPACEXTBL",
    con
)


,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


In [55]:
# Find launch sites whose names start with 'CCA'
pd.read_sql(
    "SELECT DISTINCT Launch_Site FROM SPACEXTBL WHERE Launch_Site LIKE 'CCA%'",
    con
)


,Launch_Site
0,CCAFS LC-40
1,CCAFS SLC-40


In [58]:
# Calculate the total payload mass carried by boosters launched for NASA (CRS)
pd.read_sql(
    """
    SELECT SUM(PAYLOAD_MASS__KG_) AS total_payload_mass
    FROM SPACEXTBL
    WHERE Customer = 'NASA (CRS)'
    """,
    con
)


,total_payload_mass
0,45596


In [60]:
# This query calculates the average payload mass (in kilograms)
# carried by the Falcon 9 booster version F9 v1.1.
pd.read_sql(
    """
    SELECT AVG(PAYLOAD_MASS__KG_) AS avg_payload_mass
    FROM SPACEXTBL
    WHERE Booster_Version = 'F9 v1.1'
    """,
    con
)


,avg_payload_mass
0,2928.4


In [ ]:
# This query finds the earliest date when a booster successfully landed on a ground pad.
pd.read_sql(
    """
    SELECT MIN(Date) AS first_successful_ground_landing
    FROM SPACEXTBL
    WHERE Landing_Outcome = 'Success (ground pad)'
    """,
    con
)



,first_successful_ground_landing
0,2015-12-22


In [67]:
# This query lists booster versions that successfully landed on a drone ship
# while carrying a payload mass between 4000 and 6000 kg.
pd.read_sql(
    """
    SELECT DISTINCT Booster_Version
    FROM SPACEXTBL
    WHERE Landing_Outcome = 'Success (drone ship)'
      AND PAYLOAD_MASS__KG_ > 4000
      AND PAYLOAD_MASS__KG_ < 6000
    """,
    con
)


,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


In [70]:
# This query counts the total number of missions for each outcome (success or failure),
# providing an overview of mission performance.
pd.read_sql(
    """
    SELECT Landing_Outcome, COUNT(*) AS total_count
    FROM SPACEXTBL
    GROUP BY Landing_Outcome
    """,
    con
)


,Landing_Outcome,total_count
0,Controlled (ocean),5
1,Failure,3
2,Failure (drone ship),5
3,Failure (parachute),2
4,No attempt,21
5,No attempt,1
6,Precluded (drone ship),1
7,Success,38
8,Success (drone ship),14
9,Success (ground pad),9


In [72]:
# This query groups all landing outcomes into Success or Failure categories
# and counts the total number of missions for each category.

# Normalize the landing outcomes
df['mission_outcome'] = df['Landing_Outcome'].apply(
    lambda x: 'Success' if x is not None and 'Success' in str(x) else 'Failure'
)

# Count
df['mission_outcome'].value_counts()


mission_outcome
Success    61
Failure    40
Name: count, dtype: int64

In [75]:
# This query lists all booster versions that carried the maximum payload mass
pd.read_sql(
    """
    SELECT Booster_Version
    FROM SPACEXTBL
    WHERE PAYLOAD_MASS__KG_ = (
        SELECT MAX(PAYLOAD_MASS__KG_)
        FROM SPACEXTBL
    )
    """,
    con
)


,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


In [78]:
# This query lists all records from 2015 where the landing outcome failed on a drone ship
pd.read_sql(
    """
    SELECT
        CASE strftime('%m', Date)
            WHEN '01' THEN 'January'
            WHEN '02' THEN 'February'
            WHEN '03' THEN 'March'
            WHEN '04' THEN 'April'
            WHEN '05' THEN 'May'
            WHEN '06' THEN 'June'
            WHEN '07' THEN 'July'
            WHEN '08' THEN 'August'
            WHEN '09' THEN 'September'
            WHEN '10' THEN 'October'
            WHEN '11' THEN 'November'
            WHEN '12' THEN 'December'
        END AS MonthName,
        Landing_Outcome,
        Booster_Version,
        Launch_Site
    FROM SPACEXTBL
    WHERE Landing_Outcome = 'Failure (drone ship)'
      AND strftime('%Y', Date) = '2015';
    """,
    con
)


,MonthName,Landing_Outcome,Booster_Version,Launch_Site
0,January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [79]:
# This query counts all landing outcomes between 2010-06-04 and 2017-03-20
pd.read_sql(
    """
    SELECT Landing_Outcome, COUNT(*) AS total_count
    FROM SPACEXTBL
    WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
    GROUP BY Landing_Outcome
    ORDER BY total_count DESC;
    """,
    con
)


,Landing_Outcome,total_count
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1
